In [10]:
%matplotlib inline
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import bz2
import os
from llama_cpp import Llama
import kagglehub

from transformers import pipeline

pipe = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Download latest version
path = kagglehub.dataset_download("sayelabualigah/high-quality-financial-news-dataset-for-nlp-tasks")

print(path)

with bz2.open(path, "rt", encoding="utf-8") as f:
    df = pd.read_json(f)



Device set to use cpu


C:\Users\admin\.cache\kagglehub\datasets\sayelabualigah\high-quality-financial-news-dataset-for-nlp-tasks\versions\14


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\admin\\.cache\\kagglehub\\datasets\\sayelabualigah\\high-quality-financial-news-dataset-for-nlp-tasks\\versions\\14'